In [1]:
# Import library
import pandas as pd
import numpy as np
import json
import datetime
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

In [21]:
# # Load json data from file
# data = []
# with open('dataset/sgcircuitbreaker.json', encoding="utf8") as f:
#     for line in f:
#         data.append(json.loads(line))
        
# df = pd.DataFrame(data)

In [2]:
# Loan json data into dataframe
df = pd.read_json("C:/Users/ngwei/Desktop/sgcircuitbreaker.json", lines=True)
df.head()

,_id,created_at,id,screen_name:,lang,is_quote_status,is_retweet_status,full_text,quote_text
0,{'$oid': '5eae613c65da26ed42446310'},{'$date': {'$numberLong': '1588486453000'}},{'$numberLong': '1256829397778132992'},Ashy_roz,en,True,False,Morning Peepos @MdAnde1 @SyiqinKmz,Good morning...\n#Singapore \n#circuitbreaker ...
1,{'$oid': '5eae614b65da26ed42446311'},{'$date': {'$numberLong': '1588486468000'}},{'$numberLong': '1256829460193689601'},harshdass11,en,False,False,"There is proof inðŸ“— quraan sharif, who has c...",
2,{'$oid': '5eae616665da26ed42446312'},{'$date': {'$numberLong': '1588486496000'}},{'$numberLong': '1256829574723383296'},harshdass11,en,False,True,Is creatorðŸ¤´ of all the universeðŸŒ�\nAlmigh...,
3,{'$oid': '5eae616a65da26ed42446313'},{'$date': {'$numberLong': '1588486500000'}},{'$numberLong': '1256829592066801665'},staciechan,en,False,False,Day 26 #Covid19 #circuitbreakersg: unleashing ...,
4,{'$oid': '5eae618465da26ed42446314'},{'$date': {'$numberLong': '1588486525000'}},{'$numberLong': '1256829698048274434'},SStretchsg,en,False,False,#ð�˜¿ð�™šð�™¡ð�™žð�™«ð�™šð�™§ð�™®ð�™�ð�™žð�™™ð...,


### Things to clean:
- Checking any missing/ duplicated value
- Extract data from json object inside cell for columns `_id`, `created_at`, `id`
- Rename columns

### Text cleaning:
- For `full_text`, `quote_text` column, text preprocessing is required for sentiment analysis
- Convert to lowercase, remove noise and stopword, tokenization

Before data cleaning, create another copy of dataframe.

In [3]:
df_clean = df.copy()

#### Checking missing value

In [4]:
# Checking missing value
df_clean.isnull().sum()

_id                  0
created_at           0
id                   0
screen_name:         0
lang                 0
is_quote_status      0
is_retweet_status    0
full_text            0
quote_text           0
dtype: int64

No missing value is found.

#### Extract json data

May Refer these links to see how to extract json data
- [w3school](https://www.w3schools.com/python/python_json.asp)
- [Stack Overflow](https://stackoverflow.com/questions/42354001/python-json-object-must-be-str-bytes-or-bytearray-not-dict/42354033)

In [5]:
# Extract json object from _id column
df_clean['_id'] = df_clean['_id'].apply(lambda x: json.loads(json.dumps(x))['$oid'])

In [6]:
# Extract json object from id column
df_clean['id'] = df_clean['id'].apply(lambda x: json.loads(json.dumps(x))['$numberLong'])

In [7]:
# Extract json object from id column
df_clean['created_at'] = df_clean['created_at'].apply(lambda x: json.loads(json.dumps(x))['$date']['$numberLong'])

In [8]:
# Check value
df_clean.sample(10)

,_id,created_at,id,screen_name:,lang,is_quote_status,is_retweet_status,full_text,quote_text
7965,5eb2b972b194fb30558338c3,1588771179000,1258023625115840512,pochainvestment,en,False,True,"From ironing boards to mahjong tables, #Capita...",
4741,5eb0ebe0b194fb3055832c2b,1588653018000,1257528019280736256,jursit,en,False,True,Too many judgements everywhere when it comes t...,
8971,5eb42b25d16fd09c375d5ece,1588865822000,1258420584393867265,NacaeliY,es,False,False,Nacaeli Corporate Promotions https://t.co/swfQ...,
961,5eaec7cdb194fb3055831d67,1588512710000,1256939527307837441,GwnQX1bKHQw9aLy,zh,False,False,#ç‹—å¥´ #ç”·m #ä¸»å¥´ #cbçŠ¬ \næœ‰ä¸»äººæœƒæ”¶...,
1428,5eaefea1b194fb3055831f3a,1588526746000,1256998399280246784,__Adrna,in,True,True,"Sometimes, you genuinely forgot. Mak aku gi pa...",How come the mall security outside the bakery ...
4408,5eb0a833b194fb3055832ade,1588635692000,1257455351219326986,Ali08504815,hu,False,True,Ege Ãœniversitesi'ni Ã¶rnek al!\n@bayaredutr\n...,
2928,5eb00e89b194fb3055832516,1588596355000,1257290357395484672,sntbmt,en,False,True,One more month. We can do it. #circuitbreaker ...,
6791,5eb19ef3b194fb305583342d,1588698861000,1257720298125869057,BhgtKala,en,False,True,AlcoholðŸ¥‚is so dangerous that it devastatesð...,
47,5eae629765da26ed4244633f,1588486800000,1256830852991860736,irantandspam,en,False,True,Aunty quarantine at home too long is it ? want...,
7292,5eb1e0b3b194fb3055833622,1588715692000,1257790894926954497,marcost29,und,False,False,#SSTV #Hamradio #AmateurRadio #QRP #CW #LowCos...,


#### Checking duplicated value

In [9]:
# Checking duplicated value
df_clean.duplicated().sum()

0

No duplicate row is found.

#### Convert Unix timestamp to datetime

In [10]:
# Remove last 3 characters '000' from unix string (Should be 10 digits only)
# Convert created_at values from object to string first before it can convert to timestamp format
df_clean['created_at'] = df_clean['created_at'].apply(lambda x: x[:-3])
df_clean['created_at'] = df_clean['created_at'].astype(np.int64)

In [11]:
# Convert unix timestamp to datetime format
df_clean['created_at'] = df_clean['created_at'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%dT%H:%M:%SZ'))

In [12]:
df_clean['created_at'] = pd.to_datetime(df_clean['created_at'])

#### Drop Columns
`_id` is id from mongo database, thus, it is not required in this project and could be dropped.

In [13]:
df_clean.drop('_id', axis =1, inplace =True)

#### Rename Columns
- Change `id` to `user_id`  
- Change `screen_name:` to `username`
- Change `lang` to `language`

In [14]:
df_clean.rename(columns = {'id':'user_id', 'screen_name:':'username', 'lang':'language'}, inplace = True)

#### Define Data Cleaning Functions
For tweet cleaning, may refer [here](https://towardsdatascience.com/extracting-twitter-data-pre-processing-and-sentiment-analysis-using-python-3-0-7192bd8b47cf).

In [15]:
# Convert all characters to lowercase
def to_lowercase(text):
    text  = text.lower()
    return text

In [16]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [17]:
# Remove Emoji
def remove_emoji(text):
    text = emoji_pattern.sub(r'', text)
    return text

In [18]:
# Remove Mentions
def remove_mention(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    return text

In [19]:
# Remove Hashtag
def remove_hashtag(text):
    text = re.sub(r'#\w+', '', text)
    return text

In [20]:
# Remove URL
def remove_url(text):
    text = re.sub(r'\b(?:(?:https?|ftp)://)?\w[\w-]*(?:\.[\w-]+)+\S*', ' ', text)
    return text

In [21]:
# Replace consecutive non-ASCII characters with a space
def replace_nonASCII(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    return text

In [22]:
# Remove punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [23]:
# Remove stopwords
stopwordsList = set(stopwords.words('english')) 
def remove_stopwords(text):
    text = [word for word in text if word not in stopwordsList]
    return text

In [24]:
# Tokenize words
def tokenize(text):
    word_tokens = word_tokenize(text)
    return word_tokens

In [25]:
# Combine all the functions
def datapreprocessing(review):
    
    # Convert the text into lowercase
    review = to_lowercase(review)
    
    # Clean tweet
    review = remove_mention(review)
    review = remove_url(review)
    review = remove_hashtag(review)
    review = remove_emoji(review)
    review = replace_nonASCII(review)
        
    # Remove punctuation
    review = remove_punct(review)
    
    # Tokenization
    review = word_tokenize(review)
    
    # Remove stopwords
    review = remove_stopwords(review)
    
    #return review
    return " ".join(review)

Before text preprocessing, we would like to remain all the text in **English** only.

In [26]:
df_clean = df_clean.loc[df_clean['language'] == 'en']

In [27]:
cols = ['full_text', 'quote_text']

for c in cols:
    df_clean[c] = df_clean[c].apply(lambda x: datapreprocessing(x))

In [28]:
df_clean.sample(10)

,created_at,user_id,username,language,is_quote_status,is_retweet_status,full_text,quote_text
6662,2020-05-05 17:05:08+00:00,1257717980848365569,Ahibaran__pal,en,False,True,drinking alcohol sin life drinker never happy cb,
2387,2020-05-04 06:22:56+00:00,1257193975909249031,SStretchsg,en,False,False,week wish dedicate page helping work tirelessl...,
4629,2020-05-05 03:19:35+00:00,1257510226133241857,BrandingInAsia,en,False,False,covid campaign singapore calls people emerge s...,
2427,2020-05-04 06:43:01+00:00,1257199030179782656,Ikhmalll,en,False,True,one month,
6392,2020-05-05 16:11:14+00:00,1257704417567969281,RajuOnkare,en,False,True,alcohol humans drug turns human monster cb,
4553,2020-05-05 02:10:29+00:00,1257492833285500929,nulzzee_,en,False,True,one month,
3461,2020-05-04 15:43:39+00:00,1257335085763829760,dasi_santosh,en,False,True,brahma vishnu mahesh immortal also birth death...,
6066,2020-05-05 15:20:57+00:00,1257691761289330689,RameshK53762424,en,False,True,everyone perishable brahmaloka earth cb,
6636,2020-05-05 17:03:36+00:00,1257717596717105152,ONLYDASSMALIK,en,False,True,children whose fathers intoxicated natural qua...,
7864,2020-05-06 11:15:26+00:00,1257992364964151303,Susansavanah,en,False,False,may pmwhat know masks coronavirus,


In [29]:
#df_clean.to_csv('dataset/data_clean.csv', index = False)
df_clean.to_csv('C:/Users/ngwei/Desktop/sgcircuitbreaker.csv', index = False)
